<a href="https://colab.research.google.com/github/rhymeyang/C2CPlus/blob/master/tools/chrome/ChromHeadless.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Headless Chrome and API

+ [Use Google Chrome v59 Headless mode on CentOS 7.](http://akai-tsuki.hatenablog.com/entry/2017/06/18/000000)
+ [Headless Chrome and API](https://thief.one/2018/03/06/1/)

## 相关资源

以上演示了使用命令行的方式操作headless chrome，那么怎么在代码中使用它呢？
api工具如下：

+ 官方：[puppeteer](https://github.com/GoogleChrome/puppeteer)
+ 底层：chrome-remote-interface
+ 活跃：[chromeless](https://github.com/graphcool/chromeless)
+ 非官方：[headless-chrome-crawler](https://github.com/yujiosaka/headless-chrome-crawler)

Python相关的API：

+ [pychrome](https://github.com/fate0/pychrome)
+ [Pyppeteer 推荐](https://github.com/miyakogi/pyppeteer)
+ [chromote](https://github.com/iiSeymour/chromote)
+ [chrome_remote_interface_python](https://github.com/wasiher/chrome_remote_interface_python)


## setting

### chrome.repo

/etc/yum.repos.d/chrome.repo

```config
[google-chrome]
name=google-chrome
baseurl=http://dl.google.com/linux/chrome/rpm/stable/$basearch
enabled=1
gpgcheck=1
gpgkey=https://dl-ssl.google.com/linux/linux_signing_key.pub
```


### 源管理


```shell
yum repolist
yum-config-manager

yum-config-manager --add-repo http://www.example.com/example.repo

yum-config-manager --disable XXX\*
```


### 安装


```shell
yum info google-chrome-stable

名称    ：google-chrome-stable
架构    ：x86_64
版本    ：67.0.3396.87
发布    ：1
大小    ：50 M
源    ：google-chrome/x86_64
简介    ： Google Chrome
网址    ：https://chrome.google.com/
协议    ： Multiple, see https://chrome.google.com/
描述    ： The web browser from Google
         : 
         : Google Chrome is a browser that combines a minimal design with sophisticated technology to make
         : the web faster, safer, and easier.


yum install -y google-chrome-stable
```

## Install mesa-libOSMesa and fonts

```shell
yum install -y mesa-libOSMesa mesa-libOSMesa-devel gnu-free-sans-fonts
yum install -y ipa-gothic-fonts ipa-pgothic-fonts

find / -name "libOSMesa*" -type f
/usr/lib64/libOSMesa.so.8.0.0

ln -s /usr/lib64/libOSMesa.so.8.0.0 /opt/google/chrome/libosmesa.so

```

## use headless mode

google-chrome --headless --print-to-pdf https://www.taobao.com

>```
google-chrome --headless --print-to-pdf https://www.taobao.com
[0613/113852.518653:ERROR:zygote_host_impl_linux.cc(88)] Running as root without --no-sandbox is not supported. See https://crbug.com/638180.
```

### 解决


+  /opt/google/chrome/google-chrome
+ 找到 `exec -a "$0" "$HERE/chrome" "$@"`
+ 改为 `exec -a "$0" "$HERE/chrome" "$@" --user-data-dir --no-sandbox`

截屏

    google-chrome --headless --screenshot --window-size=1024,2000 --hide-scrollbars http://akai-tsuki.hatenablog.com/
    # ls -1 screenshot.png
    screenshot.png

## Headless Chrome 基础用法



### 访问一个网页获取源码

`–dump-dom` 标志将打印 document.body.innerHTML 到标准输出：

    google-chrome --headless --disable-gpu --dump-dom https://thief.one/

### 访问一个网页将源码创建成一个PDF

`–print-to-pdf` 标志将页面转出为 PDF 文件：

    google-chrome --headless --disable-gpu --print-to-pdf https://thief.one/

### 访问一个网页并截图

使用`–screenshot`标志运行 Headless Chrome 将在当前工作目录中生成一个名为 `screenshot.png`的文件：

    chrome --headless --disable-gpu --screenshot https://thief.one/
    # 设置图片大小
    google-chrome --headless --disable-gpu --screenshot --window-size=1280,1696 https://thief.one/

### 访问一个网页并进行js交互（REPL模式）

`–repl` 标志可以使 Headless Chrome 运行在一个你可以使用浏览器评估 JS 表达式的模式下。执行下面的命令：

    google-chrome --headless --disable-gpu --repl https://thief.one
    
    >>> location.href
    {"result":{"type":"string","value":"https://thief.one"}}
    >>> quit
    
>实际使用不了，进入交互界面，一直有键盘输入，不知是否是系统问题？

### 启动一个监听端口

    google-chrome --remote-debugging-port=9222
    
    google-chrome --headless --disable-gpu --remote-debugging-port=9222

可以通过浏览器打开：http://127.0.0.1:9222

>+ google-chrome --remote-debugging-port=9222
```
(google-chrome:13677): Gtk-WARNING **: cannot open display: 
[root@mainCentOS test]# [0613/121028.659089:ERROR:nacl_helper_linux.cc(310)] NaCl helper process running without a sandbox!
Most likely you need to configure your SUID sandbox correctly
```

## 工具介绍


### puppeteer 介绍

Puppeteer 是一个由 Chrome 团队开发的 Node 库。它提供了一个高层次的 API 来控制无需显示版（或 完全版）的 Chrome。它与其他自动化测试库，如 Phantom 和 NightmareJS 相类似，但是只适用于最新版本的 Chrome。

puppeteer 安装

```shell

# 创建一个项目目录
mkdir puppeteer_test && cd puppeteer_test
npm init -y
npm i --save puppeteer

```

>+ 安装puppeteer前需要在系统上安装nodejs与npm；安装完puppeteer，默认会自动安装最新版本的chromium。
+ 系统默认安装的npm与nodejs版本都很低，而使用puppeteer需要node6.4.0+，async/await需要node7.6.0+，因此建议安装node7.6.0版本，否则会导致无法使用。

#### puppeteer 使用

在使用puppeteer前，先要确定puppeteer、nodejs、npm安装成功（版本正确），且headless chrome安装成功。

[官方API文档](https://github.com/GoogleChrome/puppeteer/blob/master/docs/api.md)

#### 获取页面的屏幕截图：

创建一个example.js文件，写入：

```js
const puppeteer = require('puppeteer');
(async() => {
const browser = await puppeteer.launch({
    headless: true,
    args: ['--no-sandbox', '--disable-setuid-sandbox'],
});
const page = await browser.newPage();
await page.goto('https://thief.one', {waitUntil: 'networkidle2'});
await page.pdf({path: 'screen.pdf', format: 'A4'});
browser.close();
})();
```

	
    node example.js

>在运行puppeteer之前不需要额外开启一个headless-chrome进程，因为其本身就会去开启。

### pyppeteer
pyppeteer模版是对puppeteer的python封装，因为puppeteer是用nodejs写的，所以要在python中使用得用pyppeteer模块。

pyppeteer安装
pyppeteer模版只支持python3.5以上版本。

1
python3 -m pip install pyppeteer
pyppeteer简单的例子
截图：

```python
import asyncio
from pyppeteer import launch
async def main():
    browser = await launch(args=['--no-sandbox'])
    page = await browser.newPage()
    await page.goto('http://example.com')
    await page.screenshot({'path': 'example.png'})
    await browser.close()
asyncio.get_event_loop().run_until_complete(main())
```

>说明：在使用pyppeteer时，不需要额外开启headless-chrome进程（与puppeteer一样）。
>
>更多pyppeteer模版使用方法，参考：https://miyakogi.github.io/pyppeteer/reference.html#page-class

### chrome-remote-interface工具

可以用来分析渲染一个页面过程中所有的请求过程，包括获取所有的请求接口以及响应内容等。再运行chrome-remote-interface代码前，需要先启动headless chrome进程，即：

    chrome --headless --remote-debugging-port=9222

安装chrome-remote-interface：


    npm install chrome-remote-interface

然后编写代码：(以获取所有请求url为例)

```js
const CDP = require('chrome-remote-interface');
// node nmask.js https://nmask.cn
var options = process.argv;
var target_url = options[2];
CDP((client) => {
    // extract domains
    const {Network, Page} = client;
    
    // setup handlers
    Network.requestWillBeSent((params) => {
        console.log(params.request.url);
    });
    Page.loadEventFired(() => {
        client.close();
    });
    
    // enable events then start!
    Promise.all([
        Network.enable(),
        Page.enable()
    ]).then(() => {
        return Page.navigate({url: target_url});//输出请求的url
    }).catch((err) => {
        console.error(err);
        client.close();
    });
}).on('error', (err) => {
    console.error(err);
});
```

运行这段代码：

    node nmask.js https://thief.one
